<a href="https://colab.research.google.com/github/ryandoyle5401/OS_Project1/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
%%writefile proj1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
int main() {

// TODO
// Create a child process for each file
// Read all files, keeping track of how often a specific word appears
// Have functionality to measure time, cpu usage, memory usage (use getrusage)
// Note: first run this using only the 7 child processes. Then run with each
//       process creating at least 2 threads to read the file at different
//       points in parallel.
// For each process, create at least two threads for parallel word counting.
// Use IPC (pipes or shared memory) so the child can communicate their word count
//       to the parent process.

  // Keeps track of how often a specific word appears
  int counter = 0;

  // Store paths to all 7 files
  char *file_paths[] = {"/content/drive/MyDrive/Colab Notebooks/txtfiles/bib",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper1", "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper2",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/progc", "/content/drive/MyDrive/Colab Notebooks/txtfiles/progl",
    "/content/drive/MyDrive/Colab Notebooks/txtfiles/progp", "/content/drive/MyDrive/Colab Notebooks/txtfiles/trans"};

  // Total number of child processes
  int NUM_OF_PROCESSES = 7;
  pid_t pid[7];

  // Keyword to keep track of
  char keyword[] = "the";

  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    pid[i] = fork();
    if (pid[i] == -1) {
      perror("failed to create child process");
      exit(1);
    } else if (pid[i] == 0) {
      // Child process
      FILE *file_ptr;
      file_ptr = fopen(file_paths[i], "r");

      // Get file size
      fseek(file_ptr, 0, SEEK_END);
      long file_size = ftell(file_ptr);
      rewind(file_ptr);

      if (file_ptr == NULL) {
        printf("file can't be opened.\n");
        return EXIT_FAILURE;
      }

      //int counter = 0;
      char str[256];
      while (fgets(str, file_size, file_ptr) != NULL) {
        // This prints all strings in the file
        //printf("%s", s);
        if (strstr(str, keyword) != NULL) {
          counter++;
        }
      }
      fclose(file_ptr);
    }
  }

  int status;
  pid_t wpid;
  for (int i = 0; i < NUM_OF_PROCESSES; i++) {
    wpid = wait(&status);
  }

  printf("The keyword %s appears %d times", keyword, counter);
  return EXIT_SUCCESS;
}

Overwriting proj1.c


In [50]:
%%shell
gcc proj1.c -o proj1
./proj1

proj1.c: In function ‘main’:
proj1.c:34:14: warning: implicit declaration of function ‘fork’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   34 |     pid[i] = fork();
      |              ^~~~
proj1.c:69:12: warning: implicit declaration of function ‘wait’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   69 |     wpid = wait(&status);
      |            ^~~~
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smashing detected ***: terminated
*** stack smas